# Clusteren

Als eerste poging is geprobeerd om de data te verdelen in clusters, om te kijken of je zo de wolken zou kunnen onderscheiden van de vogels. Hiervoor zijn verschillende clustering algoritmen gebruikt.


### K - means

K - means is een simpel partitie algoritme die de variante in een cluster probeert te verlagen.
Resultaten:
- Als je de xyz waarden meegeeft als variabele om op te clusteren, komen er nogal scherpe lijnen tussen de clusters die niet overeenkomen met vogels en wolken. Dit kan te maken hebben met dat k-means de variantie probeerd te minimaliseren, en omdat de variantie van een vogelgroep per definitie groot is, dit nooit goed geclusterd zal worden.
- Als je alleen op DBZH waarde clustert worden de vogels wel gedetecteerd, maar zitten er ook veel randjes van regenwolken bij die ook een lage DBZH waarde hebben

In [1]:
import numpy as np
import csv
import sklearn.cluster as cluster
import shutil as su
from sklearn import metrics
import hdbscan

def kmeans(path_and_name):
    original = np.genfromtxt(path_and_name, delimiter=',')
    headless = original[1:]
    data = headless[:, 0]
    data = data.reshape(-1,1)
    print(data)
    ks = KMeans(n_clusters=3).fit(data)

    print("Labels for kmeans: ")
    #labels = ks.labels_
    labels = np.append("cluster", labels)
    print(labels)

    newdata = np.append(np.reshape(headless[:,3], (-1,1)), np.reshape(headless[:,4], (-1,1)), axis=1)
    newdata = np.append(newdata, np.reshape(headless[:,5], (-1,1)), axis=1)
    newdata = np.append(newdata, np.reshape(labels,(-1,1)), axis=1)

    np.savetxt("kmeans.csv", newdata, delimiter=',')

### DBSCAN

DBSCAN is een clustering algoritme dat probeert clusters te vinden met een hoge dichtheid. Niet alle punten hoeven bij een cluster te horen. Dit zou in theorie dus de wolken kunnen vinden als dichte clusters, en de vogels overhouden als ongeclusterd. 
Resultaten:

- DBHSCAN heeft een niet intuitieve parameter eps, voor elke scan anders is. Met sommige waarden is er totaal geen resultaat, voor sommigen iets beter.
- Voor de data van 01 10 2016 om 5:00 worden met eps=3000 de wolken aardig weggefilterd. Er blijft wat ruis over en wat vogels worden tot een wolk gerekend, maar het lijkt alsof je nog iets kan zien waar een wolk heeft gezeten.


In [4]:
def DBSCAN(filename):
    original = np.genfromtxt(filename, delimiter=',')
    headless = original[1:]
    DB = cluster.DBSCAN(eps = 500000).fit(headless)
    print (len(set(DB.labels_)))
    print (percentage_empty(DB.labels_))
    width = len(headless[0])
    length = len(headless)
    b = np.zeros((length,width+1))
    b[:,:-1] = headless
    b[:, width] = np.array(DB.labels_)
    np.savetxt("output_csv\DBSCAN.csv", b, delimiter=',')
    print ("done")

### HDBSCAN

HDBSCAN zou een nog geavanceerde versie zijn van HDBSCAN, met een meer intuitieve parameter: min_cluster_size. In ons geval levert dit nog geen betere resultaten op

In [3]:
def HDBSCAN(filename):
    original = np.genfromtxt(filename, delimiter=',')
    headless = original[1:]
    DB = hdbscan.HDBSCAN(min_cluster_size = 30).fit(headless)
    print (len(set(DB.labels_)))
    print (percentage_empty(DB.labels_))
    width = len(headless[0])
    length = len(headless)
    b = np.zeros((length,width+1))
    b[:,:-1] = headless
    b[:, width] = np.array(DB.labels_)
    np.savetxt("output_csv\HDBSCAN.csv", b, delimiter=',')
    print ("done")

# Pogingen tot supervised learing

De gegeven data is niet gelabeld, waardoor je niet echt supervised algoritmes kunt gebruiken. Wel kunnen we een poging om een intuitive labeling van een clustering algoritme of van het huidige foldtobirds algoritme te gebruiken om op andere plaatjes toe te passen

### Knn met omliggende punten
omdat een belangrijk deel van de informatie van een datapunt ligt in de informatie van de datapunten er omheen (vogels zijn verder uit elkaar dan wolken, verschillen meer in richting etc.) is geprobeerd om van elk punt de dichtbijzijnde tien punten te vinden en de varantie in deze punten uit te rekenen. Op deze data is het kNN algoritme toegepast
Resultaten:
- het vinden van de dichtbijzijnde punten duurde erg lang (kan nog beter met python implementatie) daardoor konden niet alle punten gebruikt worden
- Met een tiende van de punten was het wel te doen, alleen gaf het toepassen van van kNN algortime geen nuttige informatie. Alleen bij k=2 kwamen er uberhoupt vogels uit, en als je die visualiseerd in cloudcompare waren het grotendeels niet de juiste punten. 


In [5]:
# KAN DUS OOK BETER MET PYTHON VERSIE
def find_n_closest_points(point, data, n):
    min_points = data[0:n] #zoiets
    min_distances = []
    for min_point in min_points:
        min_distances.append(np.linalg.norm(point - min_point))

    for line in range(len(data)):
        thresh = max(min_distances)    
        thresh_index = min_distances.index(thresh)
        dist = abs(np.linalg.norm(point - data[line]))
        if dist < thresh and (data[line] not in min_points):
            min_points[thresh_index] = data[line]
            min_distances[thresh_index] = dist        
    return min_points

In [6]:
# neemt een csv file, vind voor elk punt de dichtbijzijne punten, en de varantie tussen die punten voor elke varibele
# output dit in een nieuwe csv file
def csv_with_varances(filename):
    original = np.genfromtxt(filename, delimiter=',')
    headless = original[1:]
    lenh = len(headless)
    idx = np.random.randint(lenh, size=10000)
    headless = headless[idx,:]
    total = []
    count = 0
    for points in headless:
        count = count + 1
        closest = find_n_closest_points(points, headless, 10)
        var = np.var(closest, axis=0)
        total.append(np.concatenate((points, var)))
    np.savetxt("output_csv\csv_met_variances.csv", total, delimiter=',')
    return total

### Knn zonder neighbours
Zonder de dichtbijzijnde punten heb kun je eigenlijk alleen de DBZH waarden gebruiken om te clusteren, maar mogelijk zou je ook de xyz locatie van een punt kunnen gebruiken als je twee opeenvolgende beelden gebruikt. Resultaten:
- Niet echt goed, best wel chunky en verschillende hoogsten worden amper los geclassificeerd. Dit is nu geprobeerd met de DBSCAN van 1 oktober 5:00 als training data, en 5:15 als testdata

In [7]:
# LET OP
# Deze functies zijn nogal specifiek per file, wat ze doen is de nuttige dingen er uit filteren, en eventueel de 
# classificatie wat opschonen. Dit kan natuurlijk beter in een functie waar je aangeeft wele kolommen je wil, zal ik nog even fixen
def clean(filename):
    data = np.genfromtxt(filename, delimiter=',')
    useful_columns = data[:, [0, 7, 9, 10, 11, 12 ]]
    labels = data[:, 6]
    for cluster_number in range(len(labels)):
        if labels[cluster_number] == -1:
            labels[cluster_number] = 0
        else:
            labels[cluster_number] = 1
    count = 0
    for i in labels:
        if i == 0:
            count = count + 1
    print (count)
        
    return [useful_columns, labels]

def clean_whitout_neighbours(filename, labeled = 0):
    data = np.genfromtxt(filename, delimiter=',')
    if labeled == 1:
        wh = data[1:]
        return wh[:, [0, 1, 2, 3, 4]]
    useful_columns = data[:, [0, 2, 3, 4, 5]]
    labels = data[:, 6]
    for cluster_number in range(len(labels)):
        if labels[cluster_number] == -1:
            labels[cluster_number] = 0
        else:
            labels[cluster_number] = 1
    return [useful_columns, labels]

    

def clean_unlabeled(filename):
    data = np.genfromtxt(filename, delimiter=',')
    useful_columns = data[:, [0, 6, 8, 9, 10, 11 ]]
    return useful_columns

In [8]:
# deze functie voegt de labels toe aan een csvfile, ook nogal specifiek per file, kan beter
def csv_output(filename, output):
    ori = np.genfromtxt(filename, delimiter=',')
    data = ori[1:]
    useful_columns = data[:, [2, 3, 4]]
    print (len(useful_columns))
    print (len(output))
    total = np.append(useful_columns, np.reshape(output,(-1,1)), axis = 1)
    np.savetxt("output_csv\knnshit2.csv", total, delimiter=',')
    print (total)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(training[0], training[1]) 
output_labels = neigh.predict(testing)
